# Using CRDS to Lookup and Retrieve Reference Files

_Thanks to Tyler Pauly for help figuring out how to do this._


Looking up reference files from CRDS can be done `using crds.getrecommendations()` or `crds.getreferences()`. The difference is that getreferences auto downloads them to the CRDS dir; getrecommendations just returns the filename.

Each of these functions takes as input a little dict of metadata for looking up the best reference. WHich parameters are needed will vary depending on type of reference. So:

1. Go to the CRDS website and manually find some example of the kind of reference you want to search for. For example https://jwst-crds.stsci.edu/browse/jwst_nircam_dark_0468.fits
2. On that page, expand the section for "LOOKUP PATTERNS", near the bottom of the page. It will have a little table that lists various metadata parameters. The columns labeled `META.` give the metadata parameters that must be included in the call to CRDS for that kind of reference file.
3. Create a metadata dict that provides the relevant information, and pass that to the `crds.get*` function. 

In [1]:
import crds

In [2]:
meta = {
    'meta.instrument.name': 'NIRCAM',
    'meta.instrument.detector': 'NRCA4',
    'meta.subarray.name': 'FULL',
    'meta.observation.date': '2026-01-01',
    'meta.observation.time': '12:00:00',
    'meta.exposure.noutputs': 4,
    'meta.exposure.type': 'N/A',

}

refs = crds.getrecommendations(meta, reftypes=('mask','dark'), observatory='jwst')
refs

CRDS - INFO -  Calibration SW Found: jwst 1.18.0 (/Users/mperrin/software/miniconda3/envs/stenv-py3.11-2025.05.09/lib/python3.11/site-packages/jwst-1.18.0.dist-info)


{'mask': 'jwst_nircam_mask_0084.fits', 'dark': 'jwst_nircam_dark_0334.fits'}

## Remember, CRDS results depend on context

Which file(s) are returned will depend on the CRDS context that is selected. This will typically depend on the version of the JWST pipeline that you have installed.

In [3]:
crds.get_context_name(observatory='jwst')

'jwst_1364.pmap'

In [4]:
import jwst
jwst.__version__

'1.18.0'

## Which keyword metadata are needed? 

This depends on what kind of file you're looking for. If you want to query for one particular kind, see the instructions above for checking the "LOOKUP PATTERNS" listed on the CRDS website. 

`crds` also includes a function that automatically retrieves the relevant metadata from a given FITS file. This provides the mininum set of metadata which is needed to retrieve any potential reference file for that particular science file. 

In [5]:
# Let's get some arbitrary file which we can use as an example for this.
test_filename = 'jw03254020001_02101_00002_mirimage_cal.fits'

import misc_jwst
misc_jwst.mast.get_mast_filename(test_filename)

 ALREADY DOWNLOADED: ./jw03254020001_02101_00002_mirimage_cal.fits


In [6]:
# First, load the relevant mapping
crds_context = crds.get_context_name(observatory='jwst')
pmapping = crds.get_cached_mapping(crds_context)

In [7]:
meta = pmapping.get_minimum_header(test_filename)
meta

{'META.EXPOSURE.NGROUPS': '4.0',
 'META.EXPOSURE.READPATT': 'FAST',
 'META.EXPOSURE.TYPE': 'MIR_TACQ',
 'META.INSTRUMENT.BAND': 'UNDEFINED',
 'META.INSTRUMENT.CHANNEL': 'UNDEFINED',
 'META.INSTRUMENT.CORONAGRAPH': 'UNDEFINED',
 'META.INSTRUMENT.DETECTOR': 'MIRIMAGE',
 'META.INSTRUMENT.FILTER': 'FND',
 'META.INSTRUMENT.LAMP_STATE': 'OFF',
 'META.OBSERVATION.DATE': '2024-09-12',
 'META.OBSERVATION.TIME': '01:13:20.275',
 'META.SUBARRAY.NAME': 'MASK1140',
 'META.VISIT.TSOVISIT': 'F',
 'META.VISIT.TYPE': 'PRIME_TARGETED_FIXED',
 'REFTYPE': 'UNDEFINED',
 'META.INSTRUMENT.NAME': 'MIRI'}